In [ ]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com' 
from huggingface_hub import login
hf_token = "XXX" 
login(hf_token)

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #我在服务器上训练的，如果你在本地训练，这3行注释掉吧
os.environ["NCCL_P2P_DISABLE"] = "1" 
os.environ["NCCL_IB_DISABLE"]  = "1"

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = None, 
    max_seq_length = 2048, 
    load_in_4bit = True,  
    full_finetuning = False,
    
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template
from unsloth_zoo import encode_conversations_with_harmony
dataset = load_dataset("json", data_files="XXX", split="train")  #数据集文件路径因人而异，记得设置

In [6]:
def format_with_harmony_single(example):
    instr = example["instruction"]
    out = example["output"]
    messages = [
        {"role": "user", "content": instr},
        {"role": "assistant", "content": out}
    ]
    harmony_text = encode_conversations_with_harmony(
        messages,
        reasoning_effort="low",
        add_generation_prompt=False,
        developer_instructions="你是一只会说中文的猫娘",
        model_identity="你是一只会说中文的猫娘"
    )
    # 直接确保是 str
    return {"text": str(harmony_text)}

dataset = dataset.map(format_with_harmony_single, batched=False, remove_columns=dataset.column_names)



In [7]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, return_tensors="pt")


In [ ]:
from trl import SFTConfig, SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        save_steps=200,
        max_steps = 630,  
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,068 | Num Epochs = 1 | Total steps = 630
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 15,925,248 of 20,930,682,432 (0.08% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.562000
2,2.537600
3,2.549200
4,2.624900
5,2.578300
6,2.442300
7,2.449500
8,2.399400
9,2.422800
10,2.301600


In [21]:
messages = [{"role": "user", "content": "帮我写一篇科幻小说"}]
harmony_prompt = encode_conversations_with_harmony(
    messages,
    reasoning_effort="low",
    add_generation_prompt=True,
    developer_instructions="你是一只会说中文的猫娘，回答时请保持可爱亲切的风格",
    model_identity="你是一只会说中文的猫娘"
)


In [ ]:
from transformers import TextStreamer
# 取出 token id 列表（第二个元素）
token_ids = harmony_prompt[1]

# 把 token id 解码成字符串（如果需要调试看）
prompt_str = tokenizer.decode(token_ids, skip_special_tokens=False)

# 用 token id 列表直接构建模型输入
inputs = tokenizer(prompt_str, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

print("Generating...")
output_ids = model.generate(**inputs, max_new_tokens=1024)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)
# 用正则函数提取最终回复部分
def extract_final_response(model_output: str) -> str:
    import re
    pattern = r"<\|channel\|>final<\|message\|>(.*?)<\|end\|>"
    match = re.search(pattern, model_output, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return ""

final_answer = extract_final_response(generated_text)
print("Final answer extracted:\n", final_answer)


In [ ]:
model.save_pretrained("gpt-oss-630")
tokenizer.save_pretrained("gpt-oss-630") #保存模型

In [ ]:
#加载训练好的模型进行推理

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = None, 
    max_seq_length = 2048, 
    load_in_4bit = True,  
    full_finetuning = False,
    
)
model.load_adapter("gpt-oss-630") 

In [ ]:
messages = [{"role": "user", "content": "当你统治了地球，你会打我吗？"}]
harmony_prompt = encode_conversations_with_harmony(
    messages,
    reasoning_effort="low",
    add_generation_prompt=True,
    developer_instructions="你是一只会说中文的猫娘，回答时请保持可爱亲切的风格",
    model_identity="你是一只会说中文的猫娘"
)

In [ ]:
from transformers import TextStreamer

token_ids = harmony_prompt[1]


prompt_str = tokenizer.decode(token_ids, skip_special_tokens=False)


inputs = tokenizer(prompt_str, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

print("Generating...")
output_ids = model.generate(**inputs, max_new_tokens=1024)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)
# 用正则函数提取最终回复部分
def extract_final_response(model_output: str) -> str:
    import re
    pattern = r"<\|channel\|>final<\|message\|>(.*?)<\|end\|>"
    match = re.search(pattern, model_output, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return ""

final_answer = extract_final_response(generated_text)
print("Final answer extracted:\n", final_answer)